<a href="https://colab.research.google.com/github/Lambda-School-Labs/Labs26-Citrics-DS-TeamC/blob/mtoce/notebooks/Connect_to_DB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy.types import Integer, Float, Text, String, DateTime

def create_db_uri():
    DB_FLAVOR = "postgres"
    DB_PYTHON_LIBRARY = "psycopg2"
    DB_HOST = "citrics-ds-teamc.cav8gkdxva9e.us-east-1.rds.amazonaws.com"
    DB_NAME = "postgres"
    DB_USER = "postgres"
    DB_PASS = "UM*3tAe<ns!q!nG3"
    DB_PORT = "5432"
    DB_URI = DB_FLAVOR + "+" + DB_PYTHON_LIBRARY + "://" + DB_USER + ":" + DB_PASS + "@" + DB_HOST + ":" + DB_PORT + "/" + DB_NAME
    return DB_URI
    
DB_URI = create_db_uri()

In [8]:
# CONNECTION Engine with SQLAlchemy
engine = create_engine(DB_URI, echo=True)

In [10]:
_list = ["Albany, NY", "Sacramento, CA", "Austin, TX"]
def sql_query(user_queried_citystates):
    '''
    Create a SQL query to grab only the user queried cities' data from the weather table in the DB.
    Output: subset grouped DF by month and city with only queried cities
    '''

    # GET SQL weather table
    list_length = len(user_queried_citystates)

    if list_length == 1:
        city1 = user_queried_citystates[0]
        weather = pd.read_sql(f"SELECT * FROM weather WHERE 'City_State' IN ('{city1}')", con=engine, parse_dates=['created_at', 'updated_at'])
    elif list_length == 2:
        city1 = user_queried_citystates[0]
        city2 = user_queried_citystates[1]
        weather = pd.read_sql(f"SELECT * FROM weather WHERE 'City_State' IN ('{city1}', '{city2}')", con=engine, parse_dates=['created_at', 'updated_at'])
    elif list_length == 3:
        city1 = user_queried_citystates[0]
        city2 = user_queried_citystates[1]
        city3 = user_queried_citystates[2]
        weather = pd.read_sql(f"SELECT * FROM weather WHERE 'City_State' IN ('{city1}', '{city2}', '{city3}')", con=engine, parse_dates=['created_at', 'updated_at'])
    else:
        raise Exception("Please pass a list of 1-3 City_States")
    
    print(weather.head())
    # create day, month, year columns from date_time column
    weather['date_time'] = pd.to_datetime(weather['date_time'], infer_datetime_format=True)
    weather['year'] = weather['date_time'].dt.year
    weather['month'] = weather['date_time'].dt.month
    weather['day'] = weather['date_time'].dt.day
    # params=(user_queried_citystates[0], user_queried_citystates[1], user_queried_citystates[2])
    # GROUP weather data based on month and city
    grpW = weather.groupby(['month', 'City_State'], as_index=False).mean()

    # user_input = ['Albany, NY', 'Sacramento, CA', 'Austin, TX']   # user searched list of cities
    months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']

    # Set subset of grouped df for only these city-states
    subsetW = grpW[grpW['City_State'].isin(user_queried_citystates)]

    return subsetW
# call query function to get grouped frames from DB
subsetW = sql_query(_list)

subsetW.head()

2020-10-15 19:27:12,889 INFO sqlalchemy.engine.base.Engine select version()
2020-10-15 19:27:12,890 INFO sqlalchemy.engine.base.Engine {}
2020-10-15 19:27:12,918 INFO sqlalchemy.engine.base.Engine select current_schema()
2020-10-15 19:27:12,919 INFO sqlalchemy.engine.base.Engine {}
2020-10-15 19:27:12,947 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-10-15 19:27:12,948 INFO sqlalchemy.engine.base.Engine {}
2020-10-15 19:27:12,963 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-10-15 19:27:12,964 INFO sqlalchemy.engine.base.Engine {}
2020-10-15 19:27:12,979 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2020-10-15 19:27:12,981 INFO sqlalchemy.engine.base.Engine {}
2020-10-15 19:27:13,009 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
20

,month,City_State,year,day


In [ ]:
# create line chart with color var as the city-state
fig = px.line(subsetW, x='month', y='FeelsLikeF', 
            labels={'FeelsLikeF': 'Temperature (\N{DEGREE SIGN}F)', 'month': 'Month'}, 
            color='City_State', title=f'Monthly Average Temperature for Queried Cities').for_each_trace(lambda t: t.update(name=t.name.split("=")[1]))
# fig.layout.title.Font("Balto")
# fig.update_layout(tickmode='array',tickvals=months)
# fig.update_layout(legend=dict(yanchor="top",y=0.99,xanchor="left",x=0.01))    # legend inside graph top left
fig.update_layout(legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1), # legend above graph top right
              xaxis = dict(
                  tickmode = 'array',
                  tick0 = 1,
                  dtick = 1,
                  tickvals = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
                  ticktext = months
              ))   # ticks for every month
# fig.show()
temp_json = fig.to_json()

In [11]:
weather_sql = pd.read_sql(
    "SELECT * FROM weather",
    con=engine,
    parse_dates=[
        'created_at',
        'updated_at'
    ]
)

2020-10-15 19:28:09,346 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2020-10-15 19:28:09,349 INFO sqlalchemy.engine.base.Engine {'name': 'SELECT * FROM weather'}
2020-10-15 19:28:09,391 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM weather
2020-10-15 19:28:09,393 INFO sqlalchemy.engine.base.OptionEngine {}


In [45]:
weather_sql.head()

,id,pop,City_State,MaxTempF,FeelsLikeF,MinTempF,Precip_mm,TotalSnow_cm,UVindex,Humidity,WindSpeed_kmph,date_time
0,0,8336817,"New York City, NY",33.8,19.4,21.2,0.0,0.0,2,44,16,2015-01-01
1,0,8336817,"New York City, NY",39.2,26.6,28.4,0.0,0.0,1,65,13,2015-01-02
2,0,8336817,"New York City, NY",37.4,26.6,26.6,6.5,0.0,1,80,10,2015-01-03
3,0,8336817,"New York City, NY",46.4,39.2,37.4,0.9,0.0,2,95,12,2015-01-04
4,0,8336817,"New York City, NY",39.2,19.4,19.4,0.0,0.0,2,59,22,2015-01-05


In [42]:
query = 'SELECT * FROM weather WHERE "City_State" IN (%(city1)s)'
df = pd.read_sql(sql = query, columns = "City_State", params={"city1":_list[0]}, con=engine, parse_dates=['created_at', 'updated_at'])

2020-10-15 19:53:28,625 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2020-10-15 19:53:28,626 INFO sqlalchemy.engine.base.Engine {'name': 'SELECT * FROM weather WHERE "City_State" IN (%(city1)s)'}
2020-10-15 19:53:28,670 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM weather WHERE "City_State" IN (%(city1)s)
2020-10-15 19:53:28,671 INFO sqlalchemy.engine.base.OptionEngine {'city1': 'Albany, NY'}


In [46]:
df.head()

,id,pop,City_State,MaxTempF,FeelsLikeF,MinTempF,Precip_mm,TotalSnow_cm,UVindex,Humidity,WindSpeed_kmph,date_time
0,285,96460,"Albany, NY",82.4,69.8,57.2,0.3,0.0,6,77,6,2016-08-18
1,285,96460,"Albany, NY",26.6,10.4,17.6,0.0,0.0,2,50,18,2015-01-01
2,285,96460,"Albany, NY",28.4,17.6,21.2,7.9,1.5,2,76,12,2015-01-03
3,285,96460,"Albany, NY",39.2,28.4,30.2,3.7,0.0,1,96,15,2015-01-04
4,285,96460,"Albany, NY",33.8,8.6,12.2,0.3,0.0,1,61,29,2015-01-05


## Insert COVID table

In [48]:
# MOUNT Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [51]:
base_path = '/content/drive/My Drive/lambda_school/driftly_data/'
covid = pd.read_csv(base_path + 'covid_data/covid_merge.csv')
covid.head()

,seconds_since_epoch,tested,positive,deaths,postal,date,year,month,day
0,1588293736,19562,355,9,AK,2020-05-01 00:42:16,2020,5,1
1,1588372862,19562,364,9,AK,2020-05-01 22:41:02,2020,5,1
2,1588390979,20325,364,9,AK,2020-05-02 03:42:59,2020,5,2
3,1588445522,20325,365,9,AK,2020-05-02 18:52:02,2020,5,2
4,1588466696,21399,365,9,AK,2020-05-03 00:44:56,2020,5,3


In [54]:
covid.dtypes

seconds_since_epoch     int64
tested                  int64
positive                int64
deaths                  int64
postal                 object
date                   object
year                    int64
month                   int64
day                     int64
dtype: object

In [55]:
table_name = 'covid'

covid.to_sql(table_name, engine, if_exists='replace', index=False, chunksize=500,
            dtype={
                'seconds_since_epoch': Integer,
                'tested': Integer,
                'positive': Integer,
                'deaths': Integer,
                'postal': String,
                'date': DateTime,
                'year': Integer,
                'month': Integer,
                'day': Integer
            })

2020-10-15 21:21:55,543 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2020-10-15 21:21:55,544 INFO sqlalchemy.engine.base.Engine {'name': 'covid'}
2020-10-15 21:21:55,717 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE covid (
	seconds_since_epoch INTEGER, 
	tested INTEGER, 
	positive INTEGER, 
	deaths INTEGER, 
	postal VARCHAR, 
	date TIMESTAMP WITHOUT TIME ZONE, 
	year INTEGER, 
	month INTEGER, 
	day INTEGER
)


2020-10-15 21:21:55,718 INFO sqlalchemy.engine.base.Engine {}
2020-10-15 21:21:55,789 INFO sqlalchemy.engine.base.Engine COMMIT
2020-10-15 21:21:55,810 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-10-15 21:21:55,818 INFO sqlalchemy.engine.base.Engine INSERT INTO covid (seconds_since_epoch, tested, positive, deaths, postal, date, year, month, day) VALUES (%(seconds_since_epoch)s, %(tested)s, %(positive)s, %(deaths)s, %(postal)s, %(date)s

In [56]:
covid_sql = pd.read_sql(
    "SELECT * FROM covid",
    con=engine,
    parse_dates=[
        'created_at',
        'updated_at'
    ]
)

covid_sql.head()

2020-10-15 21:23:50,742 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2020-10-15 21:23:50,744 INFO sqlalchemy.engine.base.Engine {'name': 'SELECT * FROM covid'}
2020-10-15 21:23:50,786 INFO sqlalchemy.engine.base.OptionEngine SELECT * FROM covid
2020-10-15 21:23:50,786 INFO sqlalchemy.engine.base.OptionEngine {}


,seconds_since_epoch,tested,positive,deaths,postal,date,year,month,day
0,1588293736,19562,355,9,AK,2020-05-01 00:42:16,2020,5,1
1,1588372862,19562,364,9,AK,2020-05-01 22:41:02,2020,5,1
2,1588390979,20325,364,9,AK,2020-05-02 03:42:59,2020,5,2
3,1588445522,20325,365,9,AK,2020-05-02 18:52:02,2020,5,2
4,1588466696,21399,365,9,AK,2020-05-03 00:44:56,2020,5,3
